In [138]:
import numpy as np

# generate data

In [139]:
# In real world, you cannot learn how the data was generated. So do not rely on this function when coding your lab.

# In order to make the DEBUG more easy, I add the random seed for option 
def generate_data(dim = 10, num = 100, random_seed = -1):
    if random_seed != -1:
        np.random.seed(random_seed)
    x = np.random.normal(0, 10, [num, dim])

    if random_seed != -1:
        np.random.seed(random_seed)
    coef = np.random.uniform(-1, 1, [dim, 1])

    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)

    if random_seed != -1:
        np.random.seed(random_seed)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0

    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

In [140]:
# example
x, y, mr = generate_data(5, 100)
print(x[1:10])
print(y[1:10])
print(mr)
print(type(x))

[[ -7.52893478 -12.48311292  14.38112475   3.50241744  14.50216596]
 [ -4.74684664   6.40097095   5.04939938  -7.83526077  18.21878526]
 [ -8.39253406   9.49908244 -12.03386559   8.89105281  23.8697796 ]
 [  5.00142249   0.362758    -6.71770413 -14.3518933  -15.05313704]
 [  1.95691765  23.58808534  13.19229126  11.80020193  -4.63422303]
 [  4.96333707   4.33599903 -12.13573548  14.09264464  -4.00587278]
 [  3.08256896  23.40638943  13.49571548 -11.88832125  -2.29439311]
 [ 19.77873081 -12.93599172 -16.43250812   4.2009568   -2.26184068]
 [ 11.69043527  12.88329202   7.62140217  -1.09441335  -0.33970472]]
[[ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]
0.03
<class 'numpy.ndarray'>


# write your model class

In [141]:
# you can do anything necessary about the model
class SVM1:
    def __init__(self, dim):
        """
        You can add some other parameters, which I think is not necessary
        """
        self.dim = dim

    def fit(self, X:np.ndarray, y:np.ndarray):
        """
        Fit the coefficients via your methods
        """
        
    def predict(self, X:np.ndarray):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """

In [142]:
# you can do anything necessary about the model
class SVM2:
    def __init__(self, dim):
        """
        You can add some other parameters, which I think is not necessary
        """

    def fit(self, X:np.ndarray, y:np.ndarray):
        """
        Fit the coefficients via your methods
        """
        
    def predict(self, X:np.ndarray):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """

# construct and train your models

In [143]:
# generate data
X_data, y_data, mislabel = generate_data(dim=2, num=6) 

# split data
def random_Split_data(X: np.ndarray, y:np.ndarray, rate = 0.75, random_seed: int = -1):
    data: np.ndarray = np.hstack((X, y))
    m, n = data.shape
    if random_seed != -1:
        np.random.seed(abs(random_seed))
    np.random.shuffle(data)
    
    row_split = int(m * rate)
    X_train = data[0: row_split, 0: n - 1]
    y_train = data[0: row_split, n - 1: ]
    X_test = data[row_split: m, 0: n - 1]
    y_test = data[row_split: m, n - 1: ]
    
    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test = random_Split_data(X_data, y_data, rate=0.7)


# constrcut model and train (remember record time)
# model1 = SVM1() 
# model1.fit()


# predict and compare your results

In [144]:
# make prediction

def model_cmp(y_pre:np.ndarray, y_test:np.ndarray):
    # y should be in shape m x 1
    corr = 0
    sum = 0
    m, n = y_pre.shape
    
    for i in range(m):
        if y_pre[i][0] == y_test[i][0]:
            corr += 1
        sum += 1
    return corr/sum

# pred = model1.predict()

# compared with answer

# compare each methods

In [145]:
# Calculate the model accuracy on average

def model_accuracy_ave(model:str = '1', dim = 10, num = 100, devide_rate = 0.7, total_time = 100):   # all possible parameters

    model_list = ['1', '2']
    assert model in model_list, "unknown model: {}".format(model)

    sum = 0

    if model == '1':
        for i in range(total_time):
            X_data, y_data, mislabel = generate_data(dim, num) 
            X_train, y_train, X_test, y_test = random_Split_data(X_data, y_data, devide_rate)

            model = SVM1(dim) 
            model.fit(X_train, y_train)
            pred = model.predict(X_test)

            sum += model_cmp(pred, y_test)

    elif model == '2':
        for i in range(total_time):
            X_data, y_data, mislabel = generate_data(dim, num) 
            X_train, y_train, X_test, y_test = random_Split_data(X_data, y_data, devide_rate)

            model = SVM2(dim) 
            model.fit(X_train, y_train)
            pred = model.predict(X_test)

            sum += model_cmp(pred, y_test)


    return sum/total_time

print(model_accuracy_ave())

AttributeError: 'NoneType' object has no attribute 'shape'